In [3]:
from DALNET_WSE import DALNet_WSE

C:\Users\Naboth Demetrius\AppData\Roaming\Python\Python311\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\ProgramData\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
from torchvision import transforms
import pandas as pd
from sklearn.preprocessing import LabelEncoder



image_folder = r"F:\Research\Implementation\Dataset\ImageClef-2019-VQA-Med-Training\Train_images"

# Define the transformations: Convert the image to a tensor
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to a fixed size (e.g., 224x224)
    transforms.ToTensor(),  # Convert the image to a tensor
])

# Dataframe from the txt file
def getDataset(path):
    return pd.read_csv(path, sep='|', header=None, names=['image_id', 'question', 'answer'])

df = getDataset(r"F:\Research\Implementation\Dataset\ImageClef-2019-VQA-Med-Training\QAPairsByCategory\C3_Organ_train.txt")

max_question_length = df['question'].str.len().max()

label_encoder = LabelEncoder()
df['answer'] = label_encoder.fit_transform(df['answer'])

max_question_length

69

In [5]:
df

,image_id,question,answer
0,synpic41148,which organ is captured by this ct scan?,5
1,synpic43984,what organ system is primarily present in this...,7
2,synpic38930,which organ system is imaged?,3
3,synpic52143,the mri shows what organ system?,8
4,synpic20934,what organ system is shown in this x-ray?,8
...,...,...,...
3195,synpic35685,what organ is this mri showing?,4
3196,synpic35687,what organ system is shown in the image?,4
3197,synpic35701,what organ system is displayed in this mri?,4
3198,synpic48036,which organ system is shown in the ct scan?,7


In [6]:
classes = len(df['answer'].unique())
classes

10

In [7]:
# __init__(self, dim, word_len, num_heads, dropout, n_layers, vocab_size, device)

model = DALNet_WSE(768, max_question_length, 12, 0.3, 5, classes, "cuda")

C:\Users\Naboth Demetrius\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Naboth Demetrius\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
from Dataset.datasetTorch import MedicalVQADataset
from torch.utils.data import DataLoader

In [9]:
dataset = MedicalVQADataset(df, image_folder, transform, device='cuda')

# Create a DataLoader
batch_size = 3
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [10]:
count = 0
for batch in dataloader:
    count+=1
    if(count==2):
        break
    images, questions, answers = batch
    print(images.size(), len(questions), len(answers))

torch.Size([3, 3, 224, 224]) 3 3


In [11]:
model.to("cuda")

DALNet_WSE(
  (question_encoder): TSE(
    (WQ): Linear(in_features=768, out_features=768, bias=True)
    (WK): Linear(in_features=768, out_features=768, bias=True)
    (UQ): Linear(in_features=768, out_features=768, bias=True)
    (UK): Linear(in_features=768, out_features=768, bias=True)
    (V): Linear(in_features=768, out_features=768, bias=True)
  )
  (img_encoder): ResnetCustom(
    (model): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 

In [12]:
import torch.nn as nn 
import torch.optim as optim
import torch

In [13]:
num_epochs = 1
device = 'cuda'
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [19]:

# Training loop
for epoch in range(num_epochs):
    for images, questions, labels in dataloader:
        # Move data to GPU if available
        images = images.to(device)
        print(labels)
        labels = torch.tensor(labels).long().to(device)

        # Forward pass
        outputs = model(images, questions)
        print(type(outputs))
        # Compute loss
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


tensor([7, 6, 6], dtype=torch.int32)
hello
Sentence Embedding :  torch.Size([3, 1, 768])
torch.Size([3, 69, 768])
torch.Size([3, 69, 768]) torch.Size([3, 69, 768])
torch.Size([3, 1, 1, 768]) torch.Size([3, 1, 1, 768])
torch.Size([3, 1, 768]) torch.Size([3, 1, 768])
torch.Size([3, 69, 69])
torch.Size([3, 69, 69]) torch.Size([3, 1, 69])
torch.Size([3, 69, 768])
question embedding done


C:\Users\Naboth Demetrius\AppData\Local\Temp\ipykernel_10040\1305138791.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels).long().to(device)


image embedding done
DALs done
torch.Size([3, 74, 768])
torch.Size([3, 768, 74])
torch.Size([3, 768])
torch.Size([3, 10])
<class 'torch.Tensor'>
tensor([6, 3, 7], dtype=torch.int32)
hello
Sentence Embedding :  torch.Size([3, 1, 768])
torch.Size([3, 69, 768])
torch.Size([3, 69, 768]) torch.Size([3, 69, 768])
torch.Size([3, 1, 1, 768]) torch.Size([3, 1, 1, 768])
torch.Size([3, 1, 768]) torch.Size([3, 1, 768])
torch.Size([3, 69, 69])
torch.Size([3, 69, 69]) torch.Size([3, 1, 69])
torch.Size([3, 69, 768])
question embedding done
image embedding done
DALs done
torch.Size([3, 74, 768])
torch.Size([3, 768, 74])
torch.Size([3, 768])
torch.Size([3, 10])
<class 'torch.Tensor'>
tensor([2, 3, 7], dtype=torch.int32)
hello
Sentence Embedding :  torch.Size([3, 1, 768])
torch.Size([3, 69, 768])
torch.Size([3, 69, 768]) torch.Size([3, 69, 768])
torch.Size([3, 1, 1, 768]) torch.Size([3, 1, 1, 768])
torch.Size([3, 1, 768]) torch.Size([3, 1, 768])
torch.Size([3, 69, 69])
torch.Size([3, 69, 69]) torch.Size

In [30]:
sum(p.numel() for p in model.parameters())

137614973

tensor([2.3638e-07, 2.3775e-07, 3.1935e-07, 3.8696e-07, 3.4823e-07, 3.6214e-07,
        5.5743e-07, 1.0000e+00, 1.1104e-06, 5.8179e-07], device='cuda:0',
       grad_fn=<SelectBackward0>)